# MLFlow + WhyLabs Integration

This tutorial showcases how you can use the whylabs integration to:
* Capture data quality metrics while training a linear regression model in `mlflow`
* Extract whylogs data back into an in-memory format from the MLflow backend
* Log same data back into whylabs platform.

# Getting Started

To run this tutorial:
* Create and then activate the conda environment included using `environment.yml` file and run this notebook using this new environment `whylabs-mlflow` as kernel.
* You'll need to install pip into the conda environment using `conda install pip`

!pip install mlflow whylogs scikit-learn

# Setup
First, we want to filter out noisy warnings

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
import os
import datetime
import random
import time
import pandas as pd
import mlflow
import whylogs
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from whylogs import get_or_create_session
from dotenv import load_dotenv

DEBUG:whylogs.core.datasetprofile:No module named 'cudf'
DEBUG:whylogs.core.datasetprofile:Failed to import CudaDataFrame. Install cudf for CUDA support


In [3]:
load_dotenv()

True

In [4]:
assert whylogs.__version__ >= "0.1.13" # we need 0.1.13 or later for MLflow integration

# Enable whylogs Integration

Enable whylogs in MLflow to allow storing whylogs statistical profiles with every run. This method returns `True` if whylogs is able to patch MLflow. You might want to pass a session

In [5]:
session = get_or_create_session(path_to_config=".whylabs.yaml")
session.with_rotation_time = os.environ["ROTATION_TIME"]
whylogs.enable_mlflow(session=session)

DEBUG:whylogs.mlflow.patcher:Creating a real session for WhyLogsRun


True

# Dataset Preparation

Download and prepare the UCI wine quality dataset. We sample the test dataset further to represent batches of data produced every second.

In [6]:
data = pd.read_csv(os.environ["DATASET_URL"], sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
# Split the data into training and test sets
train, test = train_test_split(data)

In [8]:
# Relocate predicted variable "quality" to y vectors
train_x = train.drop(["quality"], axis=1).reset_index(drop=True)
test_x = test.drop(["quality"], axis=1).reset_index(drop=True)
train_y = train[["quality"]].reset_index(drop=True)
test_y = test[["quality"]].reset_index(drop=True)

In [9]:
subset_test_x = []
subset_test_y = []
num_batches = 20
for i in range(num_batches):
    indices = random.sample(range(len(test)), 5)
    subset_test_x.append(test_x.loc[indices, :])
    subset_test_y.append(test_y.loc[indices, :])

In [10]:
# Create an MLflow experiment for our demo
experiment_name = "whylogs demo"
mlflow.set_experiment(experiment_name)
model_params = {
    "alpha": 1.0,
    "l1_ratio": 0.7
}

In [11]:
lr = ElasticNet(**model_params)
lr.fit(train_x, train_y)
print("ElasticNet model (%s):" % model_params)

ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):


In [12]:
# run predictions on the batches of data we set up earlier and log whylogs data
for i in range(num_batches):
    with mlflow.start_run(run_name=f"Run {i + 1}"):
        batch = subset_test_x[i]
        predicted_output = lr.predict(batch)

        mae = mean_absolute_error(subset_test_y[i], predicted_output)
        print("Subset %.0f, mean absolute error: %s" % (i + 1, mae))

        mlflow.log_params(model_params)
        mlflow.log_metric("mae", mae)
        batch["mae"] = mae

        for k, v in model_params.items():
            batch[k] = v
        # use whylogs to log data quality metrics for the current batch

        mlflow.whylogs.log_pandas(
            batch, 
            datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=i)
        )
        time.sleep(70)

DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/nicolasroldan/Documents/Whylabs/whylogs/examples/mlflow_whylabs_example, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/nicolasroldan/Documents/Whylabs/whylogs/examples/mlflow_whylabs_example, universal_newlines=False, shell=None, istream=None)
DEBUG:whylogs.mlflow.patcher:Detect a new run ID. Previous run ID: None. New run ID: 3db5c052aefd42d1804a80aa787daf9b.
DEBUG:whylogs.mlflow.patcher:Attempting close patcher WhyLogsRun
DEBUG:whylogs.mlflow.patcher:Finished uploading all the loggers
DEBUG:whylogs.mlflow.patcher:Finished closing the session
DEBUG:whylogs.mlflow.patcher:Creating a new logger for dataset name: 2021-09-20 22:34:30.453647+00:00. Tags: {'mlflow.user': 'nicolasroldan', 'mlflow.runName': 'Run 1', 'mlflow.source.name': '/opt/anaconda3/envs/whylabs-mlflow/lib/python3.7/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mflow.experiment_id': '1', 'mfl

Subset 1, mean absolute error: 0.6301953378702091


DEBUG:whylogs.app.writers:Using tmp dir: /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmplv2wpomg
DEBUG:whylogs.app.writers:Writing logger /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmplv2wpomg/3db5c052aefd42d1804a80aa787daf9b/profile.bin's data to whylogs/3db5c052aefd42d1804a80aa787daf9b
DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmplv2wpomg/3db5c052aefd42d1804a80aa787daf9b/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs.app.writers:Successfully uploaded logger 3db5c052aefd42d1804a80aa787daf9b data to MLFlow
DEBUG:whylogs.mlflow.patcher:Closing whylogs before ending the MLFlow run
DEBUG:whylogs.mlflow.patcher:Attempting close patcher WhyLogsRun
DEBUG:whylogs.app.writers:Using tmp dir: /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmpdh0p4ylr
DEBUG:whylogs.mlflow.pat

Subset 2, mean absolute error: 0.5195228410174103


DEBUG:whylogs.app.writers:Using tmp dir: /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmprk4_03y_
DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmpi4934_jp/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs.app.writers:Writing logger /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmprk4_03y_/1a16429818f2434fbc55676f0deabd60/profile.bin's data to whylogs/1a16429818f2434fbc55676f0deabd60
DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmprk4_03y_/1a16429818f2434fbc55676f0deabd60/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs:Upload with WhyLabs API token
DEBUG:whylogs.app.writers

...

Subset 17, mean absolute error: 0.48540558219334146


DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmp1sf3to7d/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs.app.writers:Using tmp dir: /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmp7b7e5lrc
DEBUG:whylogs:Upload with WhyLabs API token
DEBUG:whylogs.app.writers:Writing logger /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmp7b7e5lrc/be6923b086d94562870cb6bee92e4cb9/profile.bin's data to whylogs/be6923b086d94562870cb6bee92e4cb9
DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmp7b7e5lrc/be6923b086d94562870cb6bee92e4cb9/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs.app.writers

Subset 18, mean absolute error: 0.37498509354707715


DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmpm5rrgi9h/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs.app.writers:Using tmp dir: /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmphmlxgnjl
DEBUG:whylogs:Upload with WhyLabs API token
DEBUG:whylogs.app.writers:Writing logger /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmphmlxgnjl/32bed5ae15d641c4bbf93ad3bd9e0072/profile.bin's data to whylogs/32bed5ae15d641c4bbf93ad3bd9e0072
DEBUG:smart_open.smart_open_lib:{'uri': '/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmphmlxgnjl/32bed5ae15d641c4bbf93ad3bd9e0072/profile.bin', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:whylogs.app.writers

KeyboardInterrupt: 